In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup 
import re

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None)

In [ ]:
cols = []
tables = pd.read_html("products_list.html")
df = tables[0]
cols = [item[-2] for item in df.columns]
df.columns = cols

def clear_string(string):
    pattern = r"\(.*\)"
    return re.sub(pattern, "", string)


products = [clear_string(product).replace("  ", " ").strip() for product in df["DESCRIPCION DEL PRODUCTO"].tolist() if type(product) == str and "total" not in product.lower() and "todos" not in product.lower()]



for product in products:
    print(product)

# FIRST WEBPAGE 

https://www.comprasparaguai.com.br/

In [93]:
URL_TEMPLATE = "https://www.comprasparaguai.com.br/busca/?q={QUERY}"


products_data = [] 

template_data = {
    "has_prime": False,
    "prime_price": "",
    "has_macedonia": False,
    "macedonia_price": "",
    "has_promo": False,
    "has_atacado_connect": False,
    "atacado_connect_price": "",
}

for index, product in enumerate(products):
    response = requests.get(URL_TEMPLATE.format(QUERY=product.replace(" ", "+")))
    product_data = {}
    product_data.update(template_data)
    if index == 0:
        product_data["has_prime"] = True
    
    product_data["query"] = product
    product_data["status_code"] = response.status_code
    if response.status_code == 200:  
        soup = BeautifulSoup(response.text, "lxml")
        items = soup.select("div.promocao-item-preco-oferta.flex.column a")
        price_box = soup.select_one(".preco-dolar strong")
        if price_box:
            product_data["price"] = price_box.text.strip()
        else:
            price_box = soup.select_one(".price-model strong")
            if price_box:
                product_data["price"] = price_box.text.strip()
            else:
                product_data["price"] = ""
        product_data["items"] = items
        product_data["items_count"] = len(items)
        for item in items:
            if item.select_one("button"):
                href = item.get("href")  
                if href:  
                    product_data["link"] = href
                    break
                else:
                    product_data["link"] = None
            else:
                href = item.get("href")  
                if href:  
                    product_data["link"] = href
                product_data["link"] = href
                print(f"Erro ao buscar o produto '{product}': {response.status_code}")
                
        products_data.append(product_data)
    else:
        print(f"Erro ao buscar o produto '{product}': {response.status_code}")


Erro ao buscar o produto 'LATTAFA BADEE AL OUD FOR GLORY 100ML': 200


In [94]:
BASE_URL = "https://www.comprasparaguai.com.br"

for data in products_data:
    try:
        if data["link"]:
            print(BASE_URL + data["link"])
            response = requests.get(BASE_URL + data["link"])
            soup = BeautifulSoup(response.text, "lxml")
            listed_items = soup.select(".promocao-produtos-item")
            for item in listed_items:
                print(item.select_one("img").get("alt"))
                if item.select_one("img[alt='Prime shop']"):
                    print("Achou prime")
                    data["has_prime"] = True
                    data["prime_price"] = item.select_one(".promocao-item-preco-oferta.flex.column strong").text.strip()
                if item.select_one("img[alt='Macedonia']"):
                    print("Achou macedonia")
                    data["has_macedonia"] = True
                    data["macedonia_price"] = item.select_one(".promocao-item-preco-oferta.flex.column strong").text.strip()
                if item.select_one("img[alt='Atacado Connect']"):
                    print("Achou atacado connect")
                    data["has_atacado_connect"] = True
                    data["atacado_connect_price"] = item.select_one(".promocao-item-preco-oferta.flex.column strong").text.strip()
    except Exception as e:
        continue
    
    print("="*100)

https://www.comprasparaguai.com.br/perfume-al-wataniah-sabah-al-ward-eau-de-parfum-feminino-100ml_44197/
Achou macedonia
Achou atacado connect
https://www.comprasparaguai.com.br/perfume-armaf-club-de-nuit-intense-eau-de-toilette-masculino-105ml_47483/
Achou macedonia
Achou atacado connect
https://www.comprasparaguai.com.br/perfume-lattafa-ajwad-pink-to-pink-eau-de-parfum-unissex-60ml_54517/
Achou macedonia
Achou atacado connect
https://www.comprasparaguai.com.br/perfume-lattafa-asad-zanzibar-eau-de-parfum-masculino-100ml_53082/
Achou macedonia
https://www.comprasparaguai.com.br/perfume-lattafa-badee-al-oud-amethyst-eau-de-parfum-unissex-100ml_51363/
Achou macedonia
https://www.comprasparaguai.com.brhttp://primeshop.com.py/
https://www.comprasparaguai.com.br/perfume-lattafa-fakhar-eau-de-parfum-masculino-100ml_48940/
Achou macedonia
Achou atacado connect
https://www.comprasparaguai.com.br/perfume-lattafa-fakhar-extrait-eau-de-parfum-unissex-100ml_50545/
Achou macedonia
Achou atacado con

In [95]:
for data in products_data:
    for key, value in data.items():
        print(f"{key}: {value}")
    print("="*100)

has_prime: True
prime_price: 
has_macedonia: True
macedonia_price: US$ 23,50
has_promo: False
has_atacado_connect: True
atacado_connect_price: US$ 23,00
query: AL WATANIAH SABAH AL WARD EDP 100ML FEM
status_code: 200
price: 
items: [<a href="/perfume-al-wataniah-sabah-al-ward-eau-de-parfum-feminino-100ml_44197/">
<button class="btn btn-blue" type="button">
                                                30 OFERTAS
                                            </button>
</a>]
items_count: 1
link: /perfume-al-wataniah-sabah-al-ward-eau-de-parfum-feminino-100ml_44197/
has_prime: False
prime_price: 
has_macedonia: True
macedonia_price: US$ 24,25
has_promo: False
has_atacado_connect: True
atacado_connect_price: US$ 23,50
query: ARMAF CLUB DE NUIT INTENSE MAN EDT 105ML
status_code: 200
price: 
items: [<a href="/perfume-armaf-club-de-nuit-intense-eau-de-toilette-masculino-105ml_47483/">
<button class="btn btn-blue" type="button">
                                                29 OFERTAS
      

In [96]:
empty_blocks

[{'has_prime': False,
  'prime_price': '',
  'has_macedonia': False,
  'macedonia_price': '',
  'has_promo': False,
  'has_atacado_connect': False,
  'atacado_connect_price': '',
  'query': 'LATTAFA AJWAD 100ML',
  'status_code': 200,
  'price': '',
  'items': [],
  'items_count': 0},
 {'has_prime': False,
  'prime_price': '',
  'has_macedonia': False,
  'macedonia_price': '',
  'has_promo': False,
  'has_atacado_connect': False,
  'atacado_connect_price': '',
  'query': 'LATTAFA KHAMRAH BLACK EDP 100ML',
  'status_code': 200,
  'price': '',
  'items': [],
  'items_count': 0},
 {'has_prime': False,
  'prime_price': '',
  'has_macedonia': False,
  'macedonia_price': '',
  'has_promo': False,
  'has_atacado_connect': False,
  'atacado_connect_price': '',
  'query': 'LATTAFA MUSAMAM OUD 100ML',
  'status_code': 200,
  'price': '',
  'items': [],
  'items_count': 0}]